In [2]:
import numpy as np

In [3]:
import tensorflow as tf

In [4]:
from tensorflow import keras

In [5]:
from tensorflow import keras

# Load the Fashion MNIST data
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Trim the training data to the first 30,000 samples
X_train_full = X_train_full[:30000]
y_train_full = y_train_full[:30000]

# Trim the test data to the first 5,000 samples
X_test = X_test[:5000]
y_test = y_test[:5000]

# Scale the data (normalize pixel values to 0-1)
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

# Split the first 5,000 samples of training data for validation
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


In [6]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A), (X[y_5_or_6], y_B))

In [7]:
(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)

In [8]:
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)

In [9]:
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)

In [10]:
tf.random.set_seed(42)
np.random.seed(42)

In [11]:

model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [12]:
from tensorflow import keras

# Build the neural network
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))

# Add 5 hidden dense layers with selu activation
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))

# Final output layer with 8 neurons (for 8 classes)
model_A.add(keras.layers.Dense(8, activation="softmax"))

# Compile the model
model_A.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=["accuracy"]
)

# Train the model on training data (X_train_A and y_train_A)
history = model_A.fit(
    X_train_A, y_train_A, epochs=5,
    validation_data=(X_valid_A, y_valid_A)
)

# Save the model
model_A.save("my_model_A.h5")


Epoch 1/5
622/622 [==============================] - 2s 3ms/step - loss: 0.7691 - accuracy: 0.7588 - val_loss: 0.4674 - val_accuracy: 0.8495
Epoch 2/5
622/622 [==============================] - 2s 3ms/step - loss: 0.4378 - accuracy: 0.8546 - val_loss: 0.3845 - val_accuracy: 0.8729
Epoch 3/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3788 - accuracy: 0.8678 - val_loss: 0.3737 - val_accuracy: 0.8707
Epoch 4/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3479 - accuracy: 0.8789 - val_loss: 0.3245 - val_accuracy: 0.8939
Epoch 5/5
622/622 [==============================] - 2s 3ms/step - loss: 0.3288 - accuracy: 0.8849 - val_loss: 0.3178 - val_accuracy: 0.8939


RuntimeError: Dirty entry flush destroy failed (file write failed: time = Fri Jun 27 17:29:09 2025
, filename = 'my_model_A.h5', file descriptor = 44, errno = 5, error message = 'Input/output error', buf = 0x55983f7e0d10, total write size = 4096, bytes this sub-write = 4096, bytes actually written = 18446744073709551615, offset = 0)

In [15]:
from tensorflow import keras

# Build the neural network
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))

# Add 5 hidden dense layers with selu activation
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))

# Final output layer with 1 neuron and softmax activation (not typical for binary, see note below)
model_B.add(keras.layers.Dense(1, activation="softmax"))

# Compile the model for binary classification
model_B.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=["accuracy"]
)

# Train the model
history = model_B.fit(
    X_train_B, y_train_B, epochs=5,
    validation_data=(X_valid_B, y_valid_B)
)


Epoch 1/5
161/161 [==============================] - 2s 9ms/step - loss: 0.1734 - accuracy: 0.5050 - val_loss: 0.0857 - val_accuracy: 0.5000
Epoch 2/5
161/161 [==============================] - 1s 5ms/step - loss: 0.0541 - accuracy: 0.5050 - val_loss: 0.0553 - val_accuracy: 0.5000
Epoch 3/5
161/161 [==============================] - 1s 5ms/step - loss: 0.0352 - accuracy: 0.5050 - val_loss: 0.0436 - val_accuracy: 0.5000
Epoch 4/5
161/161 [==============================] - 1s 4ms/step - loss: 0.0266 - accuracy: 0.5050 - val_loss: 0.0371 - val_accuracy: 0.5000
Epoch 5/5
161/161 [==============================] - 1s 3ms/step - loss: 0.0216 - accuracy: 0.5050 - val_loss: 0.0333 - val_accuracy: 0.5000


In [19]:
from tensorflow import keras

# View summary of model_B
model_B.summary()  # This will show 275,801 trainable parameters

# Clone model_A's architecture
model_A_clone = keras.models.clone_model(model_A)

# Set the weights of model_A_clone to match model_A
model_A_clone.set_weights(model_A.get_weights())

# Create model_B_on_A using all layers of model_A except the last layer
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])

# Add a new output layer with sigmoid activation for binary classification
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# Freeze all layers except the newly added output layer
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

# Check the number of trainable parameters (should be only 51)
model_B_on_A.summary()

# Compile the new model
model_B_on_A.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=["accuracy"]
)

# Train the model on binary classification data
history = model_B_on_A.fit(
    X_train_B, y_train_B, epochs=5,
    validation_data=(X_valid_B, y_valid_B)
)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_19 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_20 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_21 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_22 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_23 (Dense)             (None, 1)                

In [22]:
# Evaluate model_B
model_B.evaluate(X_test_B, y_test_B)

# Evaluate model_B_on_A
model_B_on_A.evaluate(X_test_B, y_test_B)


31/31 [==============================] - 0s 2ms/step - loss: 0.0716 - accuracy: 0.9928


[0.07161572575569153, 0.9927611351013184]